# Kyber algorithm

## Description

This algorithm uses to provide security KEM in TLS, encryption, IoT etc, OpenSSL.

## Math Problem

### Lattice-problem

The lattice problem is a class of mathematical problems related to the geometric structure of lattices, which are grids of regularly spaced points in 
n
n-dimensional space. Key problems include the Shortest Vector Problem (SVP) and the Closest Vector Problem (CVP). SVP seeks the shortest non-zero vector in a lattice, while CVP finds the lattice point nearest to a given target. These problems are foundational in cryptography, especially in constructing post-quantum cryptographic schemes due to their computational hardness. Lattice problems are challenging even for quantum computers, making them a cornerstone for secure communication in the face of advancing quantum technology.

## Functions

n = ? - param of kyber can be different<br>
q = ? - param of kyber can be different<br>
m = x^n + x^(n-1) ... x^0 - message

A = random matrix<br>
s = 'small' polynom/matrix<br>
e = 'small' polynom/matrix<br>
t = part of the secret key<br>
(A, t) = Matrix A extended by t<br>
(e1, e2) = Matrix e1 extended by e2<br>
(0, t) = Empty Matrix 0 extended by t<br>
(u, v) = cipher text<br>


### Keygen
Key gen example.<br>
Create some A, s and e. s and e are 'small' numbers:<br>

```
t=A*s+e
```

(A, t) is the public key<br>
s is the secret key

### Encrypt

Encryption has a few steps:

```
r * (A, t) = res + (e1, e2) + (0, e3) = (u, v)
```

### Decrypt

Decryption example with math termin:

```
U*s ≈ v - m
``` 

With trick we can get m by subtracting v.<br>
U*s is close to v - m.

full kyber uses complex structure then numbers

In [ ]:
import numpy as np

In [ ]:
q = 7  # Modulus
n = 3  # Polynomial degree
eta = 2  # Bound for small coefficients

In [ ]:
# Polynomial multiplication modulo (x^n + 1)
def poly_mult_mod(poly1, poly2, q, n):
    result = np.zeros(2 * n - 1, dtype=int)
    for i in range(len(poly1)):
        for j in range(len(poly2)):
            result[i + j] += poly1[i] * poly2[j]
    # Reduce modulo x^n + 1
    for i in range(n, len(result)):
        result[i - n] -= result[i]
    result = result[:n]  # Keep only degree < n
    return result % q

In [ ]:
def poly_add_mod(poly1, poly2, q):
    return (np.array(poly1) + np.array(poly2)) % q

def poly_sub_mod(poly1, poly2, q):
    return (np.array(poly1) - np.array(poly2)) % q

def encode_message(msg, q):
    return np.array([int(b) * (q // 2) for b in msg], dtype=int)  # Encodes {0, 1} -> {0, q/2}

def decode_message(poly, q):
    return np.array([(1 if coef > q // 4 and coef < 3 * q // 4 else 0) for coef in poly])

In [ ]:
a = [1, 2, 1]  # Public polynomial
s = [2, 1, -1]  # Private key polynomial s
e = [-1, 0, 1]  # Error polynomial

# Compute public key b = a * s + e (mod q)
b = poly_add_mod(poly_mult_mod(a, s, q, n), e, q)

# Sender's private polynomials
s_prime = [1, -1, 1]  # Private key polynomial 
e_prime = [0, 1, -1]  # Error polynomial 
e_double_prime = [1, -1, 0]  # Error polynomial for message encryption

# Message to encrypt
message = [1, 0, 1]
encoded_message = encode_message(message, q)

# c = (u, v)
u = poly_add_mod(poly_mult_mod(a, s_prime, q, n), e_prime, q)  # u = a * s' + e'
v = poly_add_mod(poly_mult_mod(b, s_prime, q, n), encoded_message + e_double_prime, q)  # v = b * s' + m + e''

# Simulate decryption
v_minus_us = poly_sub_mod(v, poly_mult_mod(u, s, q, n), q)  # v - u * s (mod q)
decoded_message = decode_message(v_minus_us, q)  # Decode

Public polynomial a: [1, 2, 1]
Private key s: [2, 1, -1]
Error polynomial e: [-1, 0, 1]
Public key b: [2 6 4]

Sender's private key s': [1, -1, 1]
Error polynomials e', e'' for encryption: [0, 1, -1] [1, -1, 0]
Encoded message: [3 0 3]
Ciphertext (u, v):
u: [0 1 6]
v: [4 6 3]

Message polynomial: [1, 0, 1]

Decrypted message polynomial: [2 5 4]
Decoded message: [1 0 1]


In [ ]:
# Print results
print("Public polynomial a:", a)
print("Private key s:", s)
print("Error polynomial e:", e)
print("Public key b:", b)

print("\nSender's private key s':", s_prime)
print("Error polynomials e', e'' for encryption:", e_prime, e_double_prime)
print("Encoded message:", encoded_message)
print("Ciphertext (u, v):")
print("u:", u)
print("v:", v)

print("\nMessage polynomial:", message)
print("\nDecrypted message polynomial:", v_minus_us)
print("Decoded message:", decoded_message)

In [15]:
from kyber import Kyber512
pk, sk = Kyber512.keygen()
c, key = Kyber512.enc(pk)
_key = Kyber512.dec(c, sk)
assert key == _key

print("Kyber512 works!")
print("pk:", pk)
print("sk:", sk)
print("c:", c)
print("key:", key)
print("_key:", _key)

Kyber512 works!
pk: b'"\x8c\x0f:z4\x1a`\x9e\xf8J\x93\x0e\x91\x11B\x10\x13NY\'\x16\xa7\x14\rIi\xb6P\x15A\x03:\xc9\x11\x856\xe9\xa0H,t\x8d\xf4\x8f[\xb5\x15;\x9aL\x9f\xa25Fu\x8e\x9crY\xa4\t\xa4)\xb4U\xc2zt+\x8a\x99\xd5t\x89\xe6\xf6\x12\x1c&\xa6J\xbaT\x1a\xf4T\xaa4\xa6\x0c\xd1\xb0\xfd\x8cH\xa7P\x8f\xca\x01F\xef;m\xe1\x8c`\xa7\x9b\xc3\xff\xe2p\x16%\x99\xc3\x86\x1a\x1c\x10\xc5\x0e\xf8>\xe3T\xa8\xcb\xaa\xce\x0f+j~\x95l\xb4\xcaU>\xe7\xc3\x1f(;=\x12\xc6\x9av\x8e\xd3\xc2+\x9c1#\xb2\xab\xb2\x8b\xe0D\xce3mj\xd9\xb1\x12\xf4\t-\xa7[\xc5sL9\x1cRf<\x93/\xf5_\x04\xf4\x07\xf3+\x7f\x8b\xf9\x9b\xa27\xce\x12j\xa4\xf6\'aw o\x9bAu{c_\xe9\\\x1e.\xcb\r\x05\xebc\x18\xe0N\xf5\xb3d\xfc, \xca\x86lL\x98\x0f/\xd6\x1ewL<\x89\x0b\xa8\xa7\n\xcc\xe1\x96\x8f\xebh\'_\xd7\xa1\xd0s?R\xe2\xaa\x1c\xf2nr\xfa\x83tr\x1b\xfbr\xcdU\xf6\xa9\x1e\x1a\xa6\x8f\t\x07\xf4\xaa3\x8b;\xb01\xb7F\xfeB\x17\xb4qE\xb4\x85\x111\xd2\xc4\x97\x9aCp\xf2e\xc2\xb1DTF5\xa0\x89\xad\xe8\xba\x9c_\xb8\x19\xfe\xd4\r:\xd1*G\xc2\x1e\xbfp\xae\xeb\xb5K\x1b\xb0Z\